In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

#from torchtext.datasets import Multi30k
#from torchtext.data import Field, BucketIterator

#import spacy
import numpy as np

import random
import math
import time

import torch.utils.data as data
import pandas as pd

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        #self.embedding = nn.Embedding(input_dim, emb_dim)
        
        #self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        #embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        
        embedded = self.dropout(src)
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell
    
    
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        #self.embedding = nn.Embedding(output_dim, emb_dim)
        
        #self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.rnn = nn.LSTM(output_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input_ = input_.float().unsqueeze(0)
        #print("decode input shape",input_.shape)
        #input = [1, batch size]
        
        #embedded = self.dropout(self.embedding(input))
        embedded = self.dropout(input_)
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        #prediction = self.fc_out(output.squeeze(0)).unsqueeze(-1)
        prediction = self.fc_out(output.squeeze(0))
        #print(prediction.shape)
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [3]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        #print("trg_len",trg.shape[0],"batch_size",trg.shape[1])
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        #print(input.shape)
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            #top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            #print(trg[t].shape,output.unsqueeze(-1).shape)
            input = trg[t] if teacher_force else output
        
        return outputs

In [4]:
INPUT_DIM = 1
OUTPUT_DIM = 1
ENC_EMB_DIM = 0
DEC_EMB_DIM = 0
HID_DIM = 64
#HID_DIM = 64

N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [5]:
FOLD = 2
CITY = 0
BATCHSIZE = 10000

model.load_state_dict(torch.load("fold{}-city{}-model.pt".format(FOLD,CITY)))      

<All keys matched successfully>

In [6]:
class ENV(object):
    def __init__(self,userNum,contentNum):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum)
        self.e = np.zeros(shape=contentNum)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l_edge = 0.1
        self.l_cp = 1

        self.B = np.full(shape=userNum,fill_value=15,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):

        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)

        self.r[u] = self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (torch.from_numpy(self.r),
                torch.from_numpy(self.p) , 
                torch.from_numpy(self.e),
                torch.from_numpy(self.S),
                self.l_edge,
                self.l_cp)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()


In [7]:


import time

import collections
import copy

data_path = '/home/ubuntu/data/dataset/R3009_U5_V100/'
UIT = pd.read_csv(data_path + 'UIT.csv')
UIT['day'] = UIT['time']//(60*60)
UIT

,u,i,day,time,viewtime,video_type,video_format,city,city_isp,client_ip,conn_type,device_type
0,365,3391,0,0,148,1030,101001,0,0,11807,1,2
1,203,5779,0,0,7,1030,10203,0,0,15068,1,2
2,208,4675,0,0,92,1035,10203,0,0,5375,1,2
3,159,332,0,0,56,1030,10202,0,0,5992,1,2
4,50,674,0,0,439,1030,10203,0,0,3468,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
300978,483,6831,719,2591880,34,1030,10203,0,0,10010,1,2
300979,158,8448,719,2591880,34,1030,10203,0,0,23340,1,2
300980,483,6463,719,2591940,35,1030,10203,0,0,10010,1,2
300981,158,4715,719,2591940,34,1030,10203,0,0,23340,1,2


In [8]:
contentNum = len(UIT.i.drop_duplicates())
userNum = len(UIT.u.drop_duplicates())
contentNum,userNum

(10000, 500)

In [9]:
class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = torch.zeros(size=(env.contentNum,),dtype=int)

        self.Bu = int(env.B[self.u])
        self.contentNum = env.contentNum
        self.userNum = env.userNum

        self.r , self.p , self.e, self.S,self.l_edge, self.l_cp = env.getStatus()

        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.lastAction = self.action

        self.reward = 0
        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']

    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    
    def getReward(self,lastru,lastp,ru,p,i,action,S,Bu,l_edge,l_cp,e):

        self.Rh = - self.ALPHAh * (torch.log(lastru * lastp + (1-lastru) * (1-lastp)).sum() - torch.log(ru * p + (1-ru) * (1-p)).sum())

        self.Ro =   self.BETAo * action[i] * (S[i] / Bu + ( e[i] * l_edge + ( 1-e[i] ) * l_cp ) / S[i])

        self.Rl =   self.BETAl * ( ( 1 - action[i] )  * ( l_cp - ( e[i] * l_edge + ( 1 - e[i] ) * l_cp ) ) ) / S[i]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        return  self.Rh+self.Ro+self.Rl

    def selectAction(self,env,uit,dpc,data):

        self.lastAction = self.action
        self.lastp = self.p
        self.lastr = self.r

        self.updateViewContent(uit[1])
        self.r , self.p , self.e, self.S, self.l_edge, self.l_cp = env.getStatus()
        
        self.reward = self.getReward(self.lastr[self.u],self.lastp,self.r[self.u],self.p,self.W[-1],self.lastAction,self.S,self.Bu,self.l_edge,self.l_cp,self.e)
        
        
        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.action[self.W[-1]] = 1
        
        if data != None:
            actionIndex = self.evaluate(dpc,data,self.Bu+1)
            if self.W[-1] not in actionIndex:
                actionIndex.pop() 
        #else:
        #    actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
        #    if self.W[-1] not in actionIndex:
        #        actionIndex.pop()
            for index in actionIndex:
                self.action[index] = 1

        env.updateEnv(self.u,self.action.numpy(),uit[2])

        #print(self.W[-1],np.argwhere(ue.action.numpy()==1),cacheSet)
        return self.action
    def evaluate(self,model, batch, cachesize):

        model.eval()

        with torch.no_grad():

            src = batch.permute(1,0,2).float().cuda()

            batchsize = src.shape[1]
            trg = torch.zeros((26,batchsize,1)).cuda()

            torch.cuda.empty_cache()

            output = model(src, trg, 0) #turn off teacher forcing

            output = output.squeeze(-1)

            output = output[0].cpu().numpy()

            max_item=np.argsort(-output)[0:cachesize]
            #print(max_item,list(max_item))
        return list(max_item)

In [10]:
num_episodes = 1
TARGET_UPDATE = 1
bestReward =  float("-inf")

env = ENV(userNum,contentNum)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

Frq_hour = torch.zeros((BATCHSIZE,720)).to(device)
m = 20


rewardPara = {"alpha":1,"betao":1,"betal":1}

sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)
i_episode = 0

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    if uit[1] in np.argwhere(ue.lastAction.numpy()):
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    if uit[2]>=m:
        test_iterator = Frq_hour[:,uit[2]-m:uit[2]]/Frq_hour[:,uit[2]-m:uit[2]].sum(dim=0)
        test_iterator = test_iterator.unsqueeze(2)
        #print(test_iterator.sum())
        
    else:
        test_iterator = None
    
    ue.selectAction(env,uit,model,test_iterator)

    for content in np.argwhere(ue.action.numpy()):
        Frq_hour[content,uit[2]]+=1
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if index % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index," Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sun Sep 26 18:00:58 2021 Episode: 0   Index: 0   Loss: 0.0 --
Reward: [0. 0. 0.] total reward: 0.0
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 1.0

--Time: Sun Sep 26 18:01:37 2021 Episode: 0   Index: 10000   Loss: 0.0 --
Reward: [0.43948 0.34233 0.00194] total reward: 0.78375
UEHitrate: 0.0014  edgeHitrate 0.38096 sumHitrate 0.38236  privacy: 1.05103

--Time: Sun Sep 26 18:06:32 2021 Episode: 0   Index: 20000   Loss: 0.0 --
Reward: [0.36869 0.37614 0.00267] total reward: 0.7475
UEHitrate: 0.00175  edgeHitrate 0.41843 sumHitrate 0.42018  privacy: 0.94254

--Time: Sun Sep 26 18:11:26 2021 Episode: 0   Index: 30000   Loss: 0.0 --
Reward: [0.31179 0.36191 0.00308] total reward: 0.67678
UEHitrate: 0.0023  edgeHitrate 0.40282 sumHitrate 0.40512  privacy: 0.9181

--Time: Sun Sep 26 18:16:22 2021 Episode: 0   Index: 40000   Loss: 0.0 --
Reward: [0.26125 0.35025 0.00321] total reward: 0.61471
UEHitrate: 0.00257  edgeHitrate 0.38982 sumHitrate 0.39239  privacy: 0.91442

--T

In [11]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.     , 0.38236, 0.42018, 0.40512, 0.39239, 0.38925, 0.38541,
        0.37769, 0.38107, 0.37903, 0.37834, 0.37872, 0.37832, 0.37445,
        0.37828, 0.38244, 0.37977, 0.38292, 0.38535, 0.3867 , 0.38717,
        0.38901, 0.38725, 0.38585, 0.38469, 0.38513, 0.38326, 0.38138,
        0.38142, 0.38371, 0.38742, 0.     ]),
 array([0.     , 0.0014 , 0.00175, 0.0023 , 0.00257, 0.003  , 0.00292,
        0.00284, 0.00285, 0.00293, 0.00292, 0.00291, 0.003  , 0.00321,
        0.0033 , 0.00338, 0.00339, 0.00339, 0.00339, 0.00347, 0.00344,
        0.0035 , 0.00353, 0.00358, 0.00363, 0.00367, 0.00373, 0.00379,
        0.00382, 0.00383, 0.00384, 0.     ]),
 array([0.     , 0.38096, 0.41843, 0.40282, 0.38982, 0.38625, 0.38249,
        0.37485, 0.37822, 0.3761 , 0.37542, 0.37581, 0.37532, 0.37124,
        0.37498, 0.37906, 0.37638, 0.37952, 0.38196, 0.38323, 0.38372,
        0.38551, 0.38373, 0.38226, 0.38106, 0.38146, 0.37953, 0.3776 ,
        0.37761, 0.37988, 0.38358, 0.     ]))

In [12]:
privacyReduction

array([1.     , 1.05103, 0.94254, 0.9181 , 0.91442, 0.9065 , 0.91146,
       0.91532, 0.91458, 0.91507, 0.90999, 0.90665, 0.90902, 0.9077 ,
       0.9102 , 0.91397, 0.91537, 0.91594, 0.90963, 0.90822, 0.90558,
       0.8902 , 0.89274, 0.8956 , 0.89818, 0.90032, 0.90215, 0.90417,
       0.90602, 0.90755, 0.9088 , 0.     ])

In [ ]:
num_episodes = 1
TARGET_UPDATE = 1
bestReward =  float("-inf")

env = ENV(userNum,contentNum)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

Frq_hour = torch.zeros((BATCHSIZE,720)).to(device)
m = 20


rewardPara = {"alpha":1,"betao":1,"betal":1}

sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)
i_episode = 0

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    if uit[1] in np.argwhere(ue.lastAction.numpy()):
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    if uit[2]>=m:
        test_iterator = Frq_hour[:,uit[2]-m:uit[2]]/Frq_hour[:,uit[2]-m:uit[2]].sum(dim=0)
        test_iterator = test_iterator.unsqueeze(2)
        #print(test_iterator.sum())
        
    else:
        test_iterator = None
    
    ue.selectAction(env,uit,model,test_iterator)

    for content in np.argwhere(ue.action.numpy()):
        Frq_hour[content,uit[2]]+=1
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if index % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index," Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sat Sep 25 12:27:40 2021 Episode: 0   Index: 0   Loss: 0.0 --
Reward: [0. 0. 0.] total reward: 0.0
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 4.02118

--Time: Sat Sep 25 12:28:36 2021 Episode: 0   Index: 10000   Loss: 0.0 --
Reward: [5.15663e+00 1.03760e-01 2.87000e-03] total reward: 5.26325
UEHitrate: 0.0022  edgeHitrate 0.11589 sumHitrate 0.11809  privacy: 3.80547

--Time: Sat Sep 25 12:33:02 2021 Episode: 0   Index: 20000   Loss: 0.0 --
Reward: [2.8981  0.26103 0.0031 ] total reward: 3.16224
UEHitrate: 0.0019  edgeHitrate 0.29059 sumHitrate 0.29249  privacy: 3.11271

--Time: Sat Sep 25 12:37:05 2021 Episode: 0   Index: 30000   Loss: 0.0 --
Reward: [2.0572  0.28613 0.00333] total reward: 2.34667
UEHitrate: 0.0022  edgeHitrate 0.31866 sumHitrate 0.32086  privacy: 2.75216

--Time: Sat Sep 25 12:41:16 2021 Episode: 0   Index: 40000   Loss: 0.0 --
Reward: [1.57752 0.29479 0.00335] total reward: 1.87566
UEHitrate: 0.00235  edgeHitrate 0.32822 sumHitrate 0.33057  priv

In [ ]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.     , 0.11809, 0.29249, 0.32086, 0.33057, 0.34161, 0.34766,
        0.34644, 0.35486, 0.35615, 0.35868, 0.3615 , 0.36309, 0.36084,
        0.36633, 0.37181, 0.37019, 0.37431, 0.37755, 0.38007, 0.38123,
        0.38361, 0.38238, 0.38156, 0.38084, 0.38179, 0.38024, 0.37864,
        0.379  , 0.38163, 0.38563, 0.     ]),
 array([0.     , 0.0022 , 0.0019 , 0.0022 , 0.00235, 0.00262, 0.00248,
        0.00241, 0.0024 , 0.00249, 0.00249, 0.00251, 0.00257, 0.00275,
        0.0028 , 0.00287, 0.00288, 0.00291, 0.0029 , 0.00298, 0.00297,
        0.00304, 0.00306, 0.00311, 0.00313, 0.00318, 0.00323, 0.00329,
        0.00331, 0.00332, 0.00333, 0.     ]),
 array([0.     , 0.11589, 0.29059, 0.31866, 0.32822, 0.33899, 0.34518,
        0.34402, 0.35246, 0.35366, 0.35619, 0.35899, 0.36051, 0.3581 ,
        0.36353, 0.36894, 0.3673 , 0.3714 , 0.37465, 0.37708, 0.37826,
        0.38057, 0.37932, 0.37845, 0.3777 , 0.37861, 0.37702, 0.37535,
        0.37568, 0.37831, 0.3823 , 0.     ]))

In [ ]:
privacyReduction

array([4.02118, 3.80547, 3.11271, 2.75216, 2.54478, 2.39712, 2.28372,
       2.19842, 2.13151, 2.06743, 2.00378, 1.95859, 1.92519, 1.8899 ,
       1.86248, 1.83901, 1.81177, 1.79281, 1.77253, 1.75582, 1.73785,
       1.72101, 1.70723, 1.69112, 1.67946, 1.66804, 1.65599, 1.6432 ,
       1.63297, 1.62498, 1.61867, 0.     ])

In [12]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

torch.cuda.empty_cache()

#list_size = [int(BATCHSIZE * ratio) for ratio in [0.001,0.0025,0.005,0.0075,0.01,0.025,0.05,0.075,0.1,0.25]]
list_size = [int(BATCHSIZE * ratio) for ratio in [0.0015]]
print(list_size)


[15]


In [12]:
INPUT_DIM = 1
OUTPUT_DIM = 1
ENC_EMB_DIM = 0
DEC_EMB_DIM = 0
HID_DIM = 64
#HID_DIM = 64

N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (rnn): LSTM(1, 64, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (rnn): LSTM(1, 64, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=64, out_features=1, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [13]:
optimizer = optim.Adam(model.parameters())

#TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

#criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
criterion = nn.MSELoss(reduction='sum')


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 100,929 trainable parameters


In [14]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch[0].permute(1,0,2).float().to(device)
        trg = batch[1].permute(1,0,2).float().to(device)
        
        torch.cuda.empty_cache()
        
        #print(src.shape,trg.shape)
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        #print(output.shape,trg.shape)
        #output_dim = output.shape[-1]
        
        batchsize = trg.shape[1]
        
        #output = output[1:].view(-1)
        output = output.permute(1,0,2).reshape(batchsize,-1)
        
        
        
        #trg = trg[1:].view(-1)
        trg = trg.permute(1,0,2).reshape(batchsize,-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        if i%30 == 0:
            print("train epoch_loss",epoch_loss)
        
    return epoch_loss / len(iterator)

In [15]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0].permute(1,0,2).float().cuda()
            trg = batch[1].permute(1,0,2).float().cuda()

            torch.cuda.empty_cache()
            
            output = model(src, trg, 0) #turn off teacher forcing
            #$print(output[1],trg[1])
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            batchsize = trg.shape[1]
            
            #output = output[1:].view(-1)
            output = output.permute(1,0,2).reshape(batchsize,-1)
            
            
            
            #trg = trg[1:].view(-1)
            trg = trg.permute(1,0,2).reshape(batchsize,-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
            if i%30 == 0:
                print("vali epoch_loss",epoch_loss)
    return epoch_loss / len(iterator)

In [16]:
class Dataset(data.Dataset):
    def __init__(self,data = None,src_len = 20,trg_len=26):
        self.data = data
        self.data_lengths = len(data)
        self.src_len = src_len
        self.trg_len = trg_len
    def __getitem__(self,index):
        data=self.data[index]
        src_data = data[0:self.src_len]
        trg_data = data[self.src_len:self.trg_len+self.src_len]
        return src_data,trg_data
    def __len__(self):
        return self.data_lengths

def dataset_iter(trDataX ,trDataY):
    trData = pd.concat([trDataX, trDataY], axis=1).to_numpy()
    trData = trData[:,:,np.newaxis]
    train_data, validate_data = np.split(trData, [int(.5*len(trData))])
    train_data_loader = torch.utils.data.DataLoader(dataset=Dataset(train_data,src_len = 20,trg_len=26),batch_size=BATCHSIZE)
    validate_data_loader = torch.utils.data.DataLoader(dataset=Dataset(validate_data,src_len = 20,trg_len=26),batch_size=BATCHSIZE)
    return train_data_loader,validate_data_loader

In [17]:
N_EPOCHS = 30
CLIP = 1
FOLD = 1
CITY = 0
BATCHSIZE = 40000

best_valid_loss = float('inf')

trDataX  = pd.read_csv("fold{}_city{}_trainX.csv".format(FOLD,CITY),header=None)
trDataY  = pd.read_csv("fold{}_city{}_trainY.csv".format(FOLD,CITY),header=None)
train_iterator,valid_iterator = dataset_iter(trDataX,trDataY)


In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

torch.cuda.empty_cache()

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    #train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    #valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'fold{}-city{}-model.pt'.format(FOLD,CITY))
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

train epoch_loss 7026.6591796875
train epoch_loss 21461.371819734573
train epoch_loss 21636.382810115814
train epoch_loss 21786.870680868626
vali epoch_loss 1.3750718832015991
vali epoch_loss 42.85216295719147
vali epoch_loss 83.69077098369598
vali epoch_loss 123.66925489902496
Epoch: 01 | Time: 4m 40s
	Train Loss: 224.676 | Train PPL: 37617647900842016898324570760897361728655058618676675688118049129584683898043946836016248823742464.000
	 Val. Loss: 1.351 |  Val. PPL:   3.863
train epoch_loss 1.5785493850708008
train epoch_loss 60.31514897942543
train epoch_loss 81.31142421066761
train epoch_loss 117.85134994983673
vali epoch_loss 0.5998155474662781
vali epoch_loss 18.819255888462067
vali epoch_loss 36.400178372859955
vali epoch_loss 53.12092709541321
Epoch: 02 | Time: 4m 38s
	Train Loss: 1.258 | Train PPL:   3.519
	 Val. Loss: 0.580 |  Val. PPL:   1.786
train epoch_loss 0.674651563167572
train epoch_loss 41.43838146328926
train epoch_loss 110.40423695743084
train epoch_loss 181.328957

In [19]:
INPUT_DIM = 1
OUTPUT_DIM = 1
ENC_EMB_DIM = 0
DEC_EMB_DIM = 0
HID_DIM = 64
#HID_DIM = 64

N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (rnn): LSTM(1, 64, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (rnn): LSTM(1, 64, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=64, out_features=1, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [20]:
optimizer = optim.Adam(model.parameters())

#TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

#criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
criterion = nn.MSELoss(reduction='sum')


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 100,929 trainable parameters


In [21]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch[0].permute(1,0,2).float().to(device)
        trg = batch[1].permute(1,0,2).float().to(device)
        
        torch.cuda.empty_cache()
        
        #print(src.shape,trg.shape)
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        #print(output.shape,trg.shape)
        #output_dim = output.shape[-1]
        
        batchsize = trg.shape[1]
        
        #output = output[1:].view(-1)
        output = output.permute(1,0,2).reshape(batchsize,-1)
        
        
        
        #trg = trg[1:].view(-1)
        trg = trg.permute(1,0,2).reshape(batchsize,-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        if i%30 == 0:
            print("train epoch_loss",epoch_loss)
        
    return epoch_loss / len(iterator)

In [22]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0].permute(1,0,2).float().cuda()
            trg = batch[1].permute(1,0,2).float().cuda()

            torch.cuda.empty_cache()
            
            output = model(src, trg, 0) #turn off teacher forcing
            #$print(output[1],trg[1])
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            batchsize = trg.shape[1]
            
            #output = output[1:].view(-1)
            output = output.permute(1,0,2).reshape(batchsize,-1)
            
            
            
            #trg = trg[1:].view(-1)
            trg = trg.permute(1,0,2).reshape(batchsize,-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
            if i%30 == 0:
                print("vali epoch_loss",epoch_loss)
    return epoch_loss / len(iterator)

In [23]:
class Dataset(data.Dataset):
    def __init__(self,data = None,src_len = 20,trg_len=26):
        self.data = data
        self.data_lengths = len(data)
        self.src_len = src_len
        self.trg_len = trg_len
    def __getitem__(self,index):
        data=self.data[index]
        src_data = data[0:self.src_len]
        trg_data = data[self.src_len:self.trg_len+self.src_len]
        return src_data,trg_data
    def __len__(self):
        return self.data_lengths

def dataset_iter(trDataX ,trDataY):
    trData = pd.concat([trDataX, trDataY], axis=1).to_numpy()
    trData = trData[:,:,np.newaxis]
    train_data, validate_data = np.split(trData, [int(.5*len(trData))])
    train_data_loader = torch.utils.data.DataLoader(dataset=Dataset(train_data,src_len = 20,trg_len=26),batch_size=BATCHSIZE)
    validate_data_loader = torch.utils.data.DataLoader(dataset=Dataset(validate_data,src_len = 20,trg_len=26),batch_size=BATCHSIZE)
    return train_data_loader,validate_data_loader

In [24]:


FOLD = 2
CITY = 0
BATCHSIZE = 10000

best_valid_loss = float('inf')

trDataX  = pd.read_csv("fold{}_city{}_trainX.csv".format(FOLD,CITY),header=None)
trDataY  = pd.read_csv("fold{}_city{}_trainY.csv".format(FOLD,CITY),header=None)
train_iterator,valid_iterator = dataset_iter(trDataX,trDataY)


In [25]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

torch.cuda.empty_cache()

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    #train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    #valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'fold{}-city{}-model.pt'.format(FOLD,CITY))
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

train epoch_loss 1667.7232666015625
train epoch_loss 3723.234430551529
train epoch_loss 3768.006135880947
train epoch_loss 3837.3120723068714
train epoch_loss 3856.373703852296
vali epoch_loss 0.7135035395622253
vali epoch_loss 22.004001677036285
vali epoch_loss 42.97784161567688
vali epoch_loss 65.34676241874695
vali epoch_loss 86.48709148168564
Epoch: 01 | Time: 6m 23s
	Train Loss: 29.079 | Train PPL: 4255484729634.600
	 Val. Loss: 0.712 |  Val. PPL:   2.037
train epoch_loss 0.8098439574241638
train epoch_loss 14.967001616954803
train epoch_loss 44.25553038716316
train epoch_loss 70.65797173976898
train epoch_loss 94.44342571496964
vali epoch_loss 2.4661550521850586
vali epoch_loss 76.33621072769165
vali epoch_loss 149.88958287239075
vali epoch_loss 224.83810091018677
vali epoch_loss 298.55797004699707
Epoch: 02 | Time: 6m 21s
	Train Loss: 0.769 | Train PPL:   2.157
	 Val. Loss: 2.458 |  Val. PPL:  11.677
train epoch_loss 2.551815986633301
train epoch_loss 18.213331952691078
train ep